In [1]:
import cv2
import mediapipe as mp
import numpy as np
from collections import defaultdict

2025-02-27 10:28:18.222747: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-27 10:28:18.240540: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740632298.261666  568027 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740632298.267989  568027 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 10:28:18.289983: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Initialize Mediapipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Dictionary to store historical feedback
feedback_history = defaultdict(int)

I0000 00:00:1740632301.042699  568027 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1740632301.131518  568270 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.3), renderer: llvmpipe (LLVM 15.0.7, 256 bits)


In [3]:
def calculate_angle(a, b, c):
    """Calculate the angle between three points."""
    a, b, c = np.array([a.x, a.y]), np.array([b.x, b.y]), np.array([c.x, c.y])
    ba, bc = a - b, c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.arccos(cosine_angle) * (180.0 / np.pi)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
def draw_text(image, text, position, align="left", color=(0, 0, 255), font_scale=0.5, thickness=1):
    """Draw text on the frame."""
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
    position = (20, position[1]) if align == "left" else (image.shape[1] - text_size[0] - 20, position[1])
    cv2.putText(image, text, position, font, font_scale, color, thickness, cv2.LINE_AA)

In [5]:
def analyze_squat(image, landmarks):
    """Analyze squat form and provide detailed feedback."""
    feedback, correction_tips = "", []
    color = (0, 0, 255)  # Red text

    # Key points
    hip, knee, ankle, shoulder = landmarks[mp_pose.PoseLandmark.LEFT_HIP], landmarks[mp_pose.PoseLandmark.LEFT_KNEE], \
                                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE], landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]

    # Calculate angles
    knee_angle, back_angle = calculate_angle(hip, knee, ankle), calculate_angle(shoulder, hip, knee)

    # Squat Feedback
    if knee_angle > 160:
        feedback = "Standing: Prepare to lower yourself into the squat. Keep your weight on your heels."
    elif 120 <= knee_angle <= 160:
        feedback = "Lowering: Control the descent. Ensure your knees track over your toes without extending past them."
    elif 80 <= knee_angle < 120:
        feedback = "Bottom: Good depth! Maintain a neutral spine and keep your chest up."
    elif 120 <= knee_angle <= 140:
        feedback = "Rising: Drive through your heels and keep your core engaged as you stand up."
    else:
        feedback = "Unknown phase."

    # Form Corrections
    if knee_angle < 80:
        correction_tips.append("❌ Too shallow! Lower until your thighs are at least parallel to the ground.")
    if back_angle < 45:
        correction_tips.append("⚠️ You're leaning too far forward. Engage your core to maintain an upright posture.")
    if ankle.y > knee.y:
        correction_tips.append("⚠️ Avoid letting your knees cave inward. Focus on keeping them aligned with your toes.")

    # Detailed Posture Feedback
    if abs(landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x - landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x) > 0.1:
        correction_tips.append("⚠️ Ensure both knees are aligned and moving symmetrically.")
    if hip.y < shoulder.y:
        correction_tips.append("⚠️ Keep your hips hinged back to maintain balance and proper form.")

    # Adaptive Feedback
    if feedback_history["knee_cave"] > 5:
        correction_tips.append("💡 Try a wider stance for better depth and stability.")
    if feedback_history["shallow_squat"] > 5:
        correction_tips.append("💡 Increase weight slightly to improve strength and depth.")

    # Update feedback history
    if "knees caving in" in correction_tips:
        feedback_history["knee_cave"] += 1
    if "Too shallow" in correction_tips:
        feedback_history["shallow_squat"] += 1

    # Draw Feedback
    draw_text(image, f"Squat Feedback: {feedback}", (20, 40), color=color)
    for i, tip in enumerate(correction_tips):
        draw_text(image, tip, (20, 60 + (i * 20)), color=color)

    return feedback, correction_tips


In [6]:
def analyze_pushup(image, landmarks):
    """Analyze push-up form and provide detailed feedback."""
    feedback, correction_tips = "", []
    color = (0, 0, 255)  # Red text

    # Key points
    shoulder, elbow, wrist, hip = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER], landmarks[mp_pose.PoseLandmark.LEFT_ELBOW], \
                                   landmarks[mp_pose.PoseLandmark.LEFT_WRIST], landmarks[mp_pose.PoseLandmark.LEFT_HIP]

    # Calculate elbow angle
    elbow_angle = calculate_angle(shoulder, elbow, wrist)

    # Push-up Feedback
    if elbow_angle > 160:
        feedback = "Up Position: Keep your body in a straight line. Lower your body down with control."
    elif 80 <= elbow_angle <= 120:
        feedback = "Bottom Position: Ensure your elbows are close to your body. Push back up with control."
    elif 120 <= elbow_angle < 160:
        feedback = "Rising: Maintain a straight body line. Avoid arching your back or sticking your butt up."
    else:
        feedback = "Unknown phase."

    # Form Corrections
    if elbow_angle < 80:
        correction_tips.append("❌ Incomplete push-up! Lower your chest closer to the ground.")
    if abs(shoulder.y - hip.y) > 0.1:
        correction_tips.append("⚠️ Keep your body straight. Avoid sagging or arching your back.")
    if elbow.x > wrist.x:
        correction_tips.append("⚠️ Keep your elbows tucked in, close to your body.")

    # Draw Feedback
    draw_text(image, f"Push-up Feedback: {feedback}", (20, 40), color=color)
    for i, tip in enumerate(correction_tips):
        draw_text(image, tip, (20, 60 + (i * 20)), color=color)

    return feedback, correction_tips


In [7]:
def analyze_lunge(image, landmarks):
    """Analyze lunge form and provide detailed feedback."""
    feedback, correction_tips = "", []
    color = (0, 0, 255)  # Red text

    # Key points
    hip, knee, ankle, shoulder = landmarks[mp_pose.PoseLandmark.LEFT_HIP], landmarks[mp_pose.PoseLandmark.LEFT_KNEE], \
                                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE], landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]

    # Calculate angles
    knee_angle, hip_angle = calculate_angle(hip, knee, ankle), calculate_angle(shoulder, hip, knee)

    # Lunge Feedback
    if knee_angle > 160:
        feedback = "Standing: Step forward into the lunge. Keep your core engaged."
    elif 90 <= knee_angle <= 160:
        feedback = "Lowering: Keep your front knee behind your toes. Maintain balance and control."
    elif 45 <= knee_angle < 90:
        feedback = "Bottom: Good depth! Keep your back straight and core engaged."
    elif 90 <= knee_angle <= 120:
        feedback = "Rising: Push through your front heel to stand up. Keep your movements controlled."
    else:
        feedback = "Unknown phase."

    # Form Corrections
    if knee_angle < 45:
        correction_tips.append("❌ Too shallow! Lower until your front thigh is parallel to the ground.")
    if hip_angle < 45:
        correction_tips.append("⚠️ Keep your back straight. Avoid rounding your shoulders.")
    if knee.x > ankle.x:
        correction_tips.append("⚠️ Ensure your front knee does not extend past your toes.")

    # Detailed Posture Feedback
    if abs(landmarks[mp_pose.PoseLandmark.LEFT_HIP].y - landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y) > 0.1:
        correction_tips.append("⚠️ Keep your hips level and square to the front.")
    if shoulder.y < hip.y:
        correction_tips.append("⚠️ Maintain an upright posture. Avoid leaning forward excessively.")

    # Draw Feedback
    draw_text(image, f"Lunge Feedback: {feedback}", (20, 40), color=color)
    for i, tip in enumerate(correction_tips):
        draw_text(image, tip, (20, 60 + (i * 20)), color=color)

    return feedback, correction_tips


In [8]:
def analyze_plank(image, landmarks):
    """Analyze plank form and provide detailed feedback."""
    feedback, correction_tips = "", []
    color = (0, 0, 255)  # Red text

    # Key points
    shoulder, elbow, hip = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER], landmarks[mp_pose.PoseLandmark.LEFT_ELBOW], \
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP]

    # Calculate angles
    shoulder_angle, hip_angle = calculate_angle(shoulder, elbow, hip), calculate_angle(elbow, hip, shoulder)

    # Plank Feedback
    if 160 <= shoulder_angle <= 180 and 160 <= hip_angle <= 180:
        feedback = "Good form! Keep your body in a straight line from head to heels."
    else:
        feedback = "Adjust your form to keep your body in a straight line."

    # Form Corrections
    if shoulder_angle < 160:
        correction_tips.append("❌ Keep your shoulders directly above your elbows or wrists.")
    if hip_angle < 160:
        correction_tips.append("⚠️ Keep your hips in line with your shoulders and heels. Avoid sagging or piking your hips.")
    if elbow.y > shoulder.y:
        correction_tips.append("⚠️ Maintain a neutral neck position. Look slightly ahead of your hands.")

    # Draw Feedback
    draw_text(image, f"Plank Feedback: {feedback}", (20, 40), color=color)
    for i, tip in enumerate(correction_tips):
        draw_text(image, tip, (20, 60 + (i * 20)), color=color)

    return feedback, correction_tips


In [9]:
def analyze_bicep_curl(image, landmarks):
    """Analyze bicep curl form and provide detailed feedback."""
    feedback, correction_tips = "", []
    color = (0, 0, 255)  # Red text

    # Key points
    shoulder, elbow, wrist = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER], landmarks[mp_pose.PoseLandmark.LEFT_ELBOW], \
                             landmarks[mp_pose.PoseLandmark.LEFT_WRIST]

    # Calculate elbow angle
    elbow_angle = calculate_angle(shoulder, elbow, wrist)

    # Bicep Curl Feedback
    if elbow_angle > 160:
        feedback = "Starting Position: Keep your elbows close to your body and shoulders back."
    elif 90 <= elbow_angle <= 160:
        feedback = "Lifting: Keep your elbows stationary and lift the weight with control."
    elif 30 <= elbow_angle < 90:
        feedback = "Top Position: Squeeze your biceps at the top of the movement."
    elif 90 <= elbow_angle <= 120:
        feedback = "Lowering: Control the weight as you lower it. Avoid swinging."
    else:
        feedback = "Unknown phase."

    # Form Corrections
    if elbow_angle < 30:
        correction_tips.append("❌ Keep your elbows close to your body. Avoid flaring them out.")
    if elbow_angle > 160:
        correction_tips.append("⚠️ Avoid swinging the weight. Use a controlled motion.")
    if wrist.y < elbow.y:
        correction_tips.append("⚠️ Keep your wrists straight. Avoid bending them backward.")

    # Detailed Posture Feedback
    if abs(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y - landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y) > 0.1:
        correction_tips.append("⚠️ Ensure both elbows move symmetrically and stay close to your body.")
    if shoulder.y > elbow.y:
        correction_tips.append("⚠️ Maintain good posture. Keep your shoulders back and chest open.")

    # Draw Feedback
    draw_text(image, f"Bicep Curl Feedback: {feedback}", (20, 40), color=color)
    for i, tip in enumerate(correction_tips):
        draw_text(image, tip, (20, 60 + (i * 20)), color=color)

    return feedback, correction_tips


W0000 00:00:1740632301.229357  568175 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [10]:
def process_video(video_path, exercise_type):
    """Process the video and analyze the specified exercise."""
    positive_feedback, negative_feedback, general_tips = [], [], []
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            if exercise_type == "squat":
                feedback, correction_tips = analyze_squat(frame, landmarks)
            elif exercise_type == "pushup":
                feedback, correction_tips = analyze_pushup(frame, landmarks)
            elif exercise_type == "lunge":
                feedback, correction_tips = analyze_lunge(frame, landmarks)
            elif exercise_type == "plank":
                feedback, correction_tips = analyze_plank(frame, landmarks)
            elif exercise_type == "bicep_curl":
                feedback, correction_tips = analyze_bicep_curl(frame, landmarks)

            # Update feedback lists
            if feedback:
                positive_feedback.append(feedback)
            if correction_tips:
                negative_feedback.extend(correction_tips)

        cv2.imshow('Exercise Analysis', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    # Print final detailed feedback
    print("\n==== Final Feedback Summary ====")
    print("✅ **Good Points:**")
    for feedback in set(positive_feedback):
        print(f"- {feedback}")

    print("\n❌ **Corrections Needed:**")
    for feedback in set(negative_feedback):
        print(f"- {feedback}")

    print("\n💡 **General Tips for Improvement:**")
    for tip in set(general_tips):
        print(f"- {tip}")


In [11]:
# Run the video analysis
video_path = "wrong_squat.mp4"  # Change this to your video file path
exercise_type = "squat"  # Change to "pushup", "lunge", "plank", or "bicep_curl" for other exercises
process_video(video_path, exercise_type)

W0000 00:00:1740632301.290488  568193 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740632301.383685  568192 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.



==== Final Feedback Summary ====
✅ **Good Points:**
- Unknown phase.
- Bottom: Good depth! Maintain a neutral spine and keep your chest up.
- Lowering: Control the descent. Ensure your knees track over your toes without extending past them.
- Standing: Prepare to lower yourself into the squat. Keep your weight on your heels.

❌ **Corrections Needed:**
- ❌ Too shallow! Lower until your thighs are at least parallel to the ground.
- ⚠️ You're leaning too far forward. Engage your core to maintain an upright posture.
- ⚠️ Avoid letting your knees cave inward. Focus on keeping them aligned with your toes.

💡 **General Tips for Improvement:**
